In [39]:
import pandas as pd 
from feature_selection import FeatureSelection
fs = FeatureSelection()

#### Read data 

In [46]:
df = pd.read_csv('./data/df_encoded.csv', index_col=[0])
df_before_encoding = pd.read_csv('./data/df_before_encoding.csv', index_col=[0])

# 1 - Feature sensetivity
##### First,lets run P models (model per feature). 
##### for each model lets test if the stand alon feature can generate halpfule prediction.
##### the function does the following : Run N svm model for each feature. each i's model split randomly test and train.
 #####       Returning features that return 1 on the test set.


In [41]:
n = 10
sensetive_festure_dict = fs.scan_feature_sensitivity(df=df, 
                                                     labeled_col_name = 'label', 
                                                      N_number_of_splits = n,
                                                     test_size=0.25)

print(f'The following features:\n\n{list(sensetive_festure_dict.keys())}' 
      '\n\npredict 1 at list in once')

10 iterations left ...
5 iterations left ...
The following features:

['median_household_income']

predict 1 at list in once


###### Writing chosen features to txt file


In [42]:
l = list(sensetive_festure_dict.keys())
with open('./data/sensative_features.txt', 'w') as filehandle:
    for listitem in l:
        filehandle.write('%s\n' % listitem)
        

# 2 - Feature importance 

### Lets use the feature importence that bagging trees features has to offer.
##### Since we dont optimizing for final model we using the default params.
##### The way we use the trees here is not optimal way. usuely its recomended to use target encoding foe categorical features before runing the tree 
##### (As same is happening in CART in R and lightGBM in python), here we hotencoded the features first

In [52]:
df_feature_importence = fs.generate_feature_importance(df = df , labeled_col_name = 'label')
df_feature_importence.to_csv('./data/df_feature_importence.csv')
print(f'Median importence = {df_feature_importence.importance.median():.3}')
print(f'Mean importence = {df_feature_importence.importance.mean():.3}')
print("\n\n Top 30 features \n the first chosen features from the 'feature sensitivity' (sum_accepted_Adjust,sum_accepted_Neon,sum_accepted_Vignette)\nsectione ranked high here\n\n")
df_feature_importence.head(30)


Median importence = 0.00203
Mean importence = 0.025


 Top 30 features 
 the first chosen features from the 'feature sensitivity' (sum_accepted_Adjust,sum_accepted_Neon,sum_accepted_Vignette)
sectione ranked high here




,feature,importance
8,median_household_income,0.355525
6,user_age,0.332705
3,coast,0.087405
4,fire_housing_proximity,0.066213
5,previous_policies,0.047600
0,has_fire_alarm,0.021721
1,has_burglar_alarm,0.015507
7,number_of_na,0.015064
2,portable_electronics,0.009682
11,state_CA,0.003191


# 3 - Regularization - Lasso

##### Here we runing logistic regression model with l1 regularization. 
##### Then we will take a look on the highest and the absolute highest coefficients

In [53]:
lasso_lr_coefficients  = fs.run_lasso_lr(df=df,labeled_col_name='label')
lasso_lr_coefficients.to_csv('./data/lasso_lr_coefficients.csv')

##### top highset coefficients values

In [54]:
lasso_lr_coefficients.head(10)

,feature,coefs
2,portable_electronics,0.634238
17,state_MD,0.623242
1,has_burglar_alarm,0.301965
11,state_CA,0.218410
31,product_ho6,0.159698
22,state_NY,0.134470
7,number_of_na,0.120818
23,state_OH,0.114618
6,user_age,0.003151
20,state_NM,0.000000


##### top highset absolut coefficients 

In [55]:
lasso_abs = lasso_lr_coefficients
lasso_abs.coefs = lasso_lr_coefficients.coefs.abs()
lasso_abs = lasso_abs.sort_values(by='coefs', ascending=False) 
lasso_abs.to_csv('./data/lasso_abs.csv')

In [56]:
lasso_abs.head(10)

,feature,coefs
21,state_NV,0.718581
37,card_type_credit,0.638399
2,portable_electronics,0.634238
17,state_MD,0.623242
33,square_ft_1000.0,0.525431
10,state_AZ,0.522320
16,state_IL,0.398358
27,state_TX,0.382411
1,has_burglar_alarm,0.301965
30,product_ho4,0.297444
